# 目的
（ほぼ）１秒ごとに取得されたtickerのデータセットに内在する何かしらの相関を探る。

In [ ]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy.signal import lfilter

%matplotlib inline

In [ ]:
def calc_EMA(x, N):
    """calc_EMA(x, N) -> array-like
    calculate exponential moving average (EMA)
    """
    return _calc_EMA(x, 2./(N + 1.))

def _calc_EMA(x, alpha):
    """ _calc_EMA(x, alpha) -> array-like
    Adopted from https://qiita.com/toyolab/items/6872b32d9fa1763345d8
    """
    y,_ = lfilter([alpha], [1,alpha-1], x, zi=[x[0]*(1-alpha)])
    return y

In [ ]:
def calc_EMA2(x, N):
    alpha = 2.0 / (N + 1.0)
    y = np.zeros(len(x))
    for ii in range(len(x)):
        if ii == 0:
            y[ii] = x[ii]
        else:
            y[ii] = (1. - alpha) * y[ii - 1] + alpha * x[ii]
    return y

In [ ]:
fldrpath = r"C:\Users\take9\Desktop\data_"
filelist = glob.glob(os.path.join(fldrpath, "*.csv"))
print(len(filelist))

# 全結合
使うのはLTPのみ。

In [ ]:
ltps = np.empty(0)
for index in range(len(filelist)):
    data = pd.read_csv(filelist[index], index_col=0)
    data_ = data["ltp"].values
    ltps = np.append(ltps, data_)

# EMA

In [ ]:
points = 20
points2 = 40

ema = calc_EMA(ltps, points)
ema_2 = calc_EMA2(ltps, points)
ema2 = calc_EMA(ltps, points2)

In [ ]:
plt.plot(ema[0:100])
plt.plot(ema_2[0:100], "r--")

In [ ]:
ltp_start_index = 3600*270

plt.figure(figsize=(18,18))
for ii in range(18):
    plt.subplot(6, 3, ii + 1)
    data_ = ltps[ltp_start_index + ii*100 : ltp_start_index + (ii+1)*100]
    plt.plot(data_,"o-", color="#E74C3C")
    ema_ = ema[ltp_start_index + ii*100 : ltp_start_index + (ii+1)*100]
    plt.plot(ema_, "#D4AC0D")
    ema2_ = ema2[ltp_start_index + ii*100 : ltp_start_index + (ii+1)*100]
    plt.plot(ema2_)
    ema_diff = data_ - ema_
    ax2 = plt.gca().twinx()
    ax2.plot(ema_diff, "#16A085")
    ax2.hlines(0, 100, 0, linestyles="--")
    plt.grid()
#     break
plt.tight_layout()
plt.savefig("../images/trend_start_{}_ema_{}_{}.png".format(ltp_start_index, points, points2), 
            bbox_inches="tight", pad_inches=0.2)